In [ ]:
import openai
import uuid
import time
import threading
from google.cloud import firestore

In [ ]:
# openllm start NousResearch/Nous-Hermes-llama-2-7b --port 8001 --dtype float16 --backend vllm > openllm.log 2>&1 &

In [ ]:
# openllm start NousResearch/Nous-Hermes-llama-2-7b --port 8001 --dtype float16 --backend vllm > openllm.log 2>&1 &

In [ ]:
client = openai.OpenAI(
    api_key='EMPTY',
    base_url="http://localhost:8001/v1"
)

In [ ]:
# Use the application default credentials.
db = firestore.Client()

In [ ]:
collection_event = threading.Event()

def sort_messages_by_timestamp(chat):
    if 'messages' in chat and isinstance(chat['messages'], list):
        sorted_messages = sorted(chat['messages'], key=lambda x: x['timestamp'])
        return sorted_messages
    else:
        return chat['messages']  # or an appropriate response if the structure is not as expected

def clean_llm_output_message(message):
    for line in message.splitlines():
        if len(line) < 5:
            continue
        return line
    return message # message is already clean

def map_messages_to_openai_api(messages):
    openai_messages = []
    for message in messages:
        openai_messages.append({
            'content': message['message'],
            'role': message['role']
        })
    return openai_messages

def map_messages_to_alternating_roles(messages):
    alternating_messages = []
    last_role = None
    for message in messages:
        if message['role'] != last_role:
            alternating_messages.append(message)
            last_role = message['role']
        else:
            # If the last message was from the same role, append the message to the last message
            alternating_messages[-1]['message'] += '\n' + message['message']
    return alternating_messages

def on_snapshot(doc_snapshot, changes, read_time):
    for doc in doc_snapshot:
        # Get the last message from the chat
        print(f'Doc: {doc.id}')
        chat = doc.to_dict()
        sorted_chat_messages = sort_messages_by_timestamp(chat)
        # Check if the chat is empty
        if not sorted_chat_messages:
            print('Chat is empty')
            continue
        # Check for a message from the assistant
        if sorted_chat_messages[-1]['role'] == 'assistant':
            print('Last message is from the assistant')
            continue

        # Model UUID is not available
        if 'modelUUID' not in [model.id for model in client.models.list().data]:
            print('Model UUID is not available')
            continue

        model = chat['modelUUID']

        alternating_messages = map_messages_to_alternating_roles(sorted_chat_messages)
        openai_messages = map_messages_to_openai_api(alternating_messages)
        chat_completion = client.chat.completions.create(
            model=model,  # Replace with your preferred model
            messages=openai_messages
        )

        # Add the AI response to the chat
        cleaned_message = clean_llm_output_message(chat_completion.choices[0].message.content)
        print(f'AI Response: {cleaned_message}')
        db.collection('chats').document(doc.id).update({
            'messages': firestore.ArrayUnion([{
                'role': 'assistant',
                'message': cleaned_message,
                'timestamp': int(time.time() * 1000),
                'uuid': str(uuid.uuid4()),
                'senderUUID': chat['modelUUID']
            }])
        })
    collection_event.set()

# Add the listener to your Firestore collection
chat_ref = db.collection('chats')
chat_watch = chat_ref.on_snapshot(on_snapshot)

In [ ]:
# advertise available models in `models` collection
models_ref = db.collection('models')
for model in client.models.list().data:
    models_ref.document(model.id).set({
        'uuid': model.id,
        'name': model.id,
        'created': model.created,
        'author': model.owned_by,
        'greetingMessage': 'Hello!',
        'type': 'Chatbot'
    })

# delete available models in `models` collection when closing
def delete_models():
    for model in client.models.list().data:
        models_ref.document(model.id).delete()

In [ ]:
import atexit

atexit.register(delete_models)